# **Distribution and performance**

This notebook:

1. Shows ingestion performance when dealing with different distributions

# Query 1

Creates 1 hash table and one round robin table with same structure and popolutes with same data

In [ ]:
CREATE TABLE [dbo].[Trip_round_robin]
(
	[DateID] [int] NOT NULL,
	[MedallionID] [int] NOT NULL,
	[HackneyLicenseID] [int] NOT NULL,
	[PickupTimeID] [int] NOT NULL,
	[DropoffTimeID] [int] NOT NULL,
	[PickupGeographyID] [int] NULL,
	[DropoffGeographyID] [int] NULL,
	[PickupLatitude] [float] NULL,
	[PickupLongitude] [float] NULL,
	[PickupLatLong] [varchar](50) NULL,
	[DropoffLatitude] [float] NULL,
	[DropoffLongitude] [float] NULL,
	[DropoffLatLong] [varchar](50) NULL,
	[PassengerCount] [int] NULL,
	[TripDurationSeconds] [int] NULL,
	[TripDistanceMiles] [float] NULL,
	[PaymentType] [varchar](50) NULL,
	[FareAmount] [money] NULL,
	[SurchargeAmount] [money] NULL,
	[TaxAmount] [money] NULL,
	[TipAmount] [money] NULL,
	[TollsAmount] [money] NULL,
	[TotalAmount] [money] NULL
)
WITH
(
	DISTRIBUTION = ROUND_ROBIN--,
	--CLUSTERED COLUMNSTORE INDEX
)
GO

CREATE TABLE [dbo].[Trip_hash]
(
	[DateID] [int] NOT NULL,
	[MedallionID] [int] NOT NULL,
	[HackneyLicenseID] [int] NOT NULL,
	[PickupTimeID] [int] NOT NULL,
	[DropoffTimeID] [int] NOT NULL,
	[PickupGeographyID] [int] NULL,
	[DropoffGeographyID] [int] NULL,
	[PickupLatitude] [float] NULL,
	[PickupLongitude] [float] NULL,
	[PickupLatLong] [varchar](50) NULL,
	[DropoffLatitude] [float] NULL,
	[DropoffLongitude] [float] NULL,
	[DropoffLatLong] [varchar](50) NULL,
	[PassengerCount] [int] NULL,
	[TripDurationSeconds] [int] NULL,
	[TripDistanceMiles] [float] NULL,
	[PaymentType] [varchar](50) NULL,
	[FareAmount] [money] NULL,
	[SurchargeAmount] [money] NULL,
	[TaxAmount] [money] NULL,
	[TipAmount] [money] NULL,
	[TollsAmount] [money] NULL,
	[TotalAmount] [money] NULL
)
WITH
(
	DISTRIBUTION = hash(DateId)--,
	--CLUSTERED COLUMNSTORE INDEX
)
GO

truncate table dbo.Trip_round_robin;
insert into dbo.Trip_round_robin 
select top(10000000)* from dbo.Trip
option (label = 'INSERT rr');

truncate table dbo.Trip_hash;
insert into dbo.Trip_hash
select top(10000000)* from dbo.Trip
option (label = 'INSERT hash');

DBCC PDW_SHOWSPACEUSED('dbo.Trip_round_robin')
DBCC PDW_SHOWSPACEUSED('dbo.Trip_hash')

Query 2

Shows duration and plans for inserts

In [8]:
-- Query 2
select 'hash', total_elapsed_time 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT hash'
    order by start_time desc 

select 'hash', total_elapsed_time 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT rr'
    order by start_time desc 

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT hash'
    order by start_time desc 
)

select * from sys.dm_pdw_request_steps as t 
where request_id =
(
    select top(1) request_id 
    from sys.dm_pdw_exec_requests  as t 
    where t.[label] = 'INSERT rr'
    order by start_time desc 
)



(1 row affected)

(2 rows affected)

(12 rows affected)

(5 rows affected)

Total execution time: 00:00:00.671

(No column name),total_elapsed_time
hash,162831


(No column name),total_elapsed_time
hash,18922
hash,144971


request_id,step_index,plan_node_id,operation_type,distribution_type,location_type,status,error_id,start_time,end_time,total_elapsed_time,row_count,estimated_rows,command
QID26195,0,-1,OnOperation,Unspecified,Control,Complete,NULL,2023-01-07 17:42:40.747,2023-01-07 17:42:40.760,15,-1,-1,"CREATE TABLE [tempdb].[QTables].[QTable_9c8c948544ac4af494349e9514335c60] ([DateID] INT NOT NULL, [MedallionID] INT NOT NULL, [HackneyLicenseID] INT NOT NULL, [PickupTimeID] INT NOT NULL, [DropoffTimeID] INT NOT NULL, [PickupGeographyID] INT, [DropoffGeographyID] INT, [PickupLatitude] FLOAT(53), [PickupLongitude] FLOAT(53), [PickupLatLong] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS, [DropoffLatitude] FLOAT(53), [DropoffLongitude] FLOAT(53), [DropoffLatLong] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS, [PassengerCount] INT, [TripDurationSeconds] INT, [TripDistanceMiles] FLOAT(53), [PaymentType] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS, [FareAmount] MONEY, [SurchargeAmount] MONEY, [TaxAmount] MONEY, [TipAmount] MONEY, [TollsAmount] MONEY, [TotalAmount] MONEY ) WITH(DISTRIBUTED_MOVE_FILE='');"
QID26195,1,11,PartitionMoveOperation,AllDistributions,Compute,Complete,NULL,2023-01-07 17:42:40.760,2023-01-07 17:43:54.513,73751,170261325,10000000,"SELECT [T1_1].[DateID] AS [DateID], [T1_1].[MedallionID] AS [MedallionID], [T1_1].[HackneyLicenseID] AS [HackneyLicenseID], [T1_1].[PickupTimeID] AS [PickupTimeID], [T1_1].[DropoffTimeID] AS [DropoffTimeID], [T1_1].[PickupGeographyID] AS [PickupGeographyID], [T1_1].[DropoffGeographyID] AS [DropoffGeographyID], [T1_1].[PickupLatitude] AS [PickupLatitude], [T1_1].[PickupLongitude] AS [PickupLongitude], [T1_1].[PickupLatLong] AS [PickupLatLong], [T1_1].[DropoffLatitude] AS [DropoffLatitude], [T1_1].[DropoffLongitude] AS [DropoffLongitude], [T1_1].[DropoffLatLong] AS [DropoffLatLong], [T1_1].[PassengerCount] AS [PassengerCount], [T1_1].[TripDurationSeconds] AS [TripDurationSeconds], [T1_1].[TripDistanceMiles] AS [TripDistanceMiles], [T1_1].[PaymentType] AS [PaymentType], [T1_1].[FareAmount] AS [FareAmount], [T1_1].[SurchargeAmount] AS [SurchargeAmount], [T1_1].[TaxAmount] AS [TaxAmount], [T1_1].[TipAmount] AS [TipAmount], [T1_1].[TollsAmount] AS [TollsAmount], [T1_1].[TotalAmount] AS [TotalAmount] FROM (SELECT TOP (CAST ((10000000) AS BIGINT)) [T2_1].[DateID] AS [DateID], [T2_1].[MedallionID] AS [MedallionID], [T2_1].[HackneyLicenseID] AS [HackneyLicenseID], [T2_1].[PickupTimeID] AS [PickupTimeID], [T2_1].[DropoffTimeID] AS [DropoffTimeID], [T2_1].[PickupGeographyID] AS [PickupGeographyID], [T2_1].[DropoffGeographyID] AS [DropoffGeographyID], [T2_1].[PickupLatitude] AS [PickupLatitude], [T2_1].[PickupLongitude] AS [PickupLongitude], [T2_1].[PickupLatLong] AS [PickupLatLong], [T2_1].[DropoffLatitude] AS [DropoffLatitude], [T2_1].[DropoffLongitude] AS [DropoffLongitude], [T2_1].[DropoffLatLong] AS [DropoffLatLong], [T2_1].[PassengerCount] AS [PassengerCount], [T2_1].[TripDurationSeconds] AS [TripDurationSeconds], [T2_1].[TripDistanceMiles] AS [TripDistanceMiles], [T2_1].[PaymentType] AS [PaymentType], [T2_1].[FareAmount] AS [FareAmount], [T2_1].[SurchargeAmount] AS [SurchargeAmount], [T2_1].[TaxAmount] AS [TaxAmount], [T2_1].[TipAmount] AS [TipAmount], [T2_1].[TollsAmount] AS [TollsAmount], [T2_1].[TotalAmount] AS [TotalAmount] FROM [dsqlpool].[dbo].[Trip] AS T2_1) AS T1_1 OPTION (MAXDOP 1, MIN_GRANT_PERCENT = [MIN_GRANT], DISTRIBUTED_MOVE(N''), MAX_GRANT_PERCENT = [MAX_GRANT])"
QID26195,2,-1,RandomIDOperation,Unspecified,Control,Complete,NULL,2023-01-07 17:43:54.513,2023-01-07 17:43:54.513,0,-1,-1,TEMP_ID_32
QID26195,3,-1,OnOperation,AllComputeNodes,Compute,Complete,NULL,2023-01-07 17:43:54.513,2023-01-07 17:43:54.543,31,-1,-1,"CREATE TABLE [tempdb].[dbo].[TEMP_ID_32] ([DateID] INT NOT NULL, [MedallionID] INT NOT NULL, [HackneyLicenseID] INT NOT NULL, [PickupTimeID] INT NOT NULL, [DropoffTimeID] INT NOT NULL, [PickupGeographyID] INT, [DropoffGeographyID] INT, [PickupLatitude] FLOAT(53), [PickupLon

request_id,step_index,plan_node_id,operation_type,distribution_type,location_type,status,error_id,start_time,end_time,total_elapsed_time,row_count,estimated_rows,command
QID26327,0,-1,RandomIDOperation,Unspecified,Control,Complete,NULL,2023-01-07 17:48:02.580,2023-01-07 17:48:02.580,0,-1,-1,TEMP_ID_34
QID26327,1,-1,OnOperation,AllComputeNodes,Compute,Complete,NULL,2023-01-07 17:48:02.580,2023-01-07 17:48:02.610,31,-1,-1,"CREATE TABLE [qtabledb].[dbo].[TEMP_ID_34] ([GeographyID] INT NOT NULL, [ZipCodeBKey] VARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, [County] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS, [City] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS, [State] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS, [Country] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS, [ZipCode] VARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS ) WITH(DISTRIBUTED_MOVE_FILE='');"
QID26327,2,2,BroadcastMoveOperation,AllDistributions,Compute,Complete,NULL,2023-01-07 17:48:02.610,2023-01-07 17:48:02.783,171,305179,305179,"SELECT [T1_1].[GeographyID] AS [GeographyID], [T1_1].[ZipCodeBKey] AS [ZipCodeBKey], [T1_1].[County] AS [County], [T1_1].[City] AS [City], [T1_1].[State] AS [State], [T1_1].[Country] AS [Country], [T1_1].[ZipCode] AS [ZipCode] FROM [dsqlpool].[dbo].[Geography] AS T1_1 OPTION (MAXDOP 1, MIN_GRANT_PERCENT = [MIN_GRANT], DISTRIBUTED_MOVE(N''), MAX_GRANT_PERCENT = [MAX_GRANT])"
QID26327,3,24,ReturnOperation,AllDistributions,Compute,Complete,NULL,2023-01-07 17:48:02.783,2023-01-07 17:48:21.440,18656,100000,0,"SELECT [T1_1].[DateID] AS [DateID], [T1_1].[MedallionID] AS [MedallionID], [T1_1].[HackneyLicenseID] AS [HackneyLicenseID], [T1_1].[PickupTimeID] AS [PickupTimeID], [T1_1].[DropoffTimeID] AS [DropoffTimeID], [T1_1].[PickupGeographyID] AS [PickupGeographyID], [T1_1].[DropoffGeographyID] AS [DropoffGeographyID], [T1_1].[PickupLatitude] AS [PickupLatitude], [T1_1].[PickupLongitude] AS [PickupLongitude], [T1_1].[PickupLatLong] AS [PickupLatLong], [T1_1].[DropoffLatitude] AS [DropoffLatitude], [T1_1].[DropoffLongitude] AS [DropoffLongitude], [T1_1].[DropoffLatLong] AS [DropoffLatLong], [T1_1].[PassengerCount] AS [PassengerCount], [T1_1].[TripDurationSeconds] AS [TripDurationSeconds], [T1_1].[TripDistanceMiles] AS [TripDistanceMiles], [T1_1].[PaymentType] AS [PaymentType], [T1_1].[FareAmount] AS [FareAmount], [T1_1].[SurchargeAmount] AS [SurchargeAmount], [T1_1].[TaxAmount] AS [TaxAmount], [T1_1].[TipAmount] AS [TipAmount], [T1_1].[TollsAmount] AS [TollsAmount], [T1_1].[TotalAmount] AS [TotalAmount], [T1_1].[GeographyID] AS [GeographyID], [T1_1].[ZipCodeBKey] AS [ZipCodeBKey], [T1_1].[County] AS [County], [T1_1].[City] AS [City], [T1_1].[State] AS [State], [T1_1].[Country] AS [Country], [T1_1].[ZipCode] AS [ZipCode] FROM (SELECT TOP (CAST ((100000) AS BIGINT)) [T2_2].[DateID] AS [DateID], [T2_2].[MedallionID] AS [MedallionID], [T2_2].[HackneyLicenseID] AS [HackneyLicenseID], [T2_2].[PickupTimeID] AS [PickupTimeID], [T2_2].[DropoffTimeID] AS [DropoffTimeID], [T2_2].[PickupGeographyID] AS [PickupGeographyID], [T2_2].[DropoffGeographyID] AS [DropoffGeographyID], [T2_2].[PickupLatitude] AS [PickupLatitude], [T2_2].[PickupLongitude] AS [PickupLongitude], [T2_2].[PickupLatLong] AS [PickupLatLong], [T2_2].[DropoffLatitude] AS [DropoffLatitude], [T2_2].[DropoffLongitude] AS [DropoffLongitude], [T2_2].[DropoffLatLong] AS [DropoffLatLong], [T2_2].[PassengerCount] AS [PassengerCount], [T2_2].[TripDurationSeconds] AS [TripDurationSeconds], [T2_2].[TripDistanceMiles] AS [TripDistanceMiles], [T2_2].[PaymentType] AS [PaymentType], [T2_2].[FareAmount] AS [FareAmount], [T2_2].[SurchargeAmount] AS [SurchargeAmount], [T2_2].[TaxAmount] AS [TaxAmount], [T2_2].[TipAmount] AS [TipAmount], [T2_2].[TollsAmount] AS [TollsAmount], [T2_2].[TotalAmount] AS [TotalAmount], [T2_1].[GeographyID] AS [GeographyID], [T2_1].[ZipCodeBKey] AS [ZipCodeBKey], [T2_1].[County] AS [County], [T2_1].[City] AS [City], [T2_1].[State] AS [State], [T2_1].[Count